In [6]:
from textgenrnn import textgenrnn
textgen = textgenrnn()
textgen.train_from_file('TfIdfQuestions.pkl', num_epochs=1)
textgen.generate()

92,268 texts collected.
Training on 12,230,109 character sequences.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/1
 5679/95547 [>.............................] - ETA: 7:42:24 - loss: 4.1189

KeyboardInterrupt: 

In [104]:
def lireDocs():
    docs = {}

    fichier_tree = ET.parse("corpusIrisVersion4.xml")
    fichier_docs = fichier_tree.getroot()
    compteur_docs = 0
    for document in fichier_docs:
        
        while compteur_docs <=5:
            compteur_docs += 1
            doc_id = document.attrib['id']
            questions = ''
            answers = ''
            doc_class = document.attrib['class']
            doc_subclass = document.attrib['subclass']
            for child in document:
                if child.tag == 'question':
                    questions += child.text.rstrip()

                if child.tag == 'answer':
                    answers += child.text.rstrip()

            doc = {}
            doc['id']= doc_id
            doc['class'] = doc_class
            doc['subclass'] = doc_subclass
            doc['questions'] = questions
            doc['answers'] = answers
            docs[doc_id] = doc

    print("Total docs : ", compteur_docs)

    return docs

docs = lireDocs()

Total docs :  6


In [219]:
import math
import os
from random import sample
import pickle

from scipy.spatial.distance import cdist
import tqdm
import json
import numpy as np
import indicoio
from texttable import Texttable
indicoio.config.api_key = "0393ee7c70f815f9ddd823d2343c2711"

'''
Use indico's Text Features API to find text similarity and create a customer support bot that automatically responds to FAQs from users.
Tutorial: https://indico.io/blog/faqs-bot-text-features-api/
'''


def make_feats(data):
    """
    Send our text data through the indico API and return each text example's text vector representation
    """
    chunks = [data[x:x+100] for x in range(0, len(data), 100)]
    feats = []
    # just a progress bar to show us how much we have left
    for chunk in chunks:
        feats.extend(indicoio.text_features(chunk))
    return feats

def calculate_distances(f1, f2):
    # cosine distance is the most reasonable metric for comparison of these 300d vectors
    distances = cdist(f1, f2, 'cosine')
    return distances

def similarity_text(idx, distance_matrix, data, n_similar=5):
    """
    idx: the index of the text we're looking for similar questions to
         (data[idx] corresponds to the actual text we care about)
    distance_matrix: an m by n matrix that stores the distance between
                     document m and document n at distance_matrix[m][n]
    data: a flat list of text data
    """
    t = Texttable()
    t.set_cols_width([50, 20])

    # these are the indexes of the texts that are most similar to the text at data[idx]
    # note that this list of 10 elements contains the index of text that we're comparing things to at idx 0
    sorted_distance_idxs = np.argsort(distance_matrix[idx])[:n_similar] # EX: [252, 102, 239, ...]
    # this is the index of the text that is most similar to the query (index 0)
    most_sim_idx = sorted_distance_idxs[1]

    # header for texttable
    t.add_rows([['Text', 'Similarity']])
    print(t.draw())

    # set the variable that will hold our matching FAQ
    faq_match = None

    for similar_idx in sorted_distance_idxs:
        # actual text data for display
        datum = data[similar_idx]

        # distance in cosine space from our text example to the similar text example
        distance = distance_matrix[idx][similar_idx]

        # how similar that text data is to our input example
        similarity =  1 - distance

        # add the text + the floating point similarity value to our Texttable() object for display
        t.add_rows([[datum, str(round(similarity, 2))]])
        print(t.draw())

        # set a confidence threshold
        # TODO

    # print the appropriate answer to the FAQ, or bring in a human to respond
    # TODO

def input_question(data, feats):
    # input a question
    question = input("What is your question? ")
    # add the user question and its vector representations to the corresponding lists, `data` and `feats`
    # insert them at index 0 so you know exactly where they are for later distance calculations
    if question is not None:
        data.insert(0, question)
    new_feats = indicoio.text_features(question)
    feats.insert(0, new_feats)
    return data, feats

def run():
    data = list(docs.keys())
    print("FAQ data received. Finding features.")

    feats = make_feats(data)
    with open('faq_feats.pkl', 'wb') as f:
        pickle.dump(feats, f)

    with open('faq_feats.pkl', 'rb') as f:
        feats = pickle.load(f)
        #pickle.dump(feats, f)

    print("Features found -- success! Calculating similarities...")
    #print("FAQ features found!")

    new_data, new_feats = input_question(data, feats)

    distance_matrix = calculate_distances(feats, new_feats)
    print("Similarities found. Generating table.")

    idx = 0
    similarity_text(idx, distance_matrix, new_data)
    print('\n' + '-' * 80)

if __name__ == "__main__":
    run()


FAQ data received. Finding features.
Features found -- success! Calculating similarities...
What is your question? fjeji
Similarities found. Generating table.
+----------------------------------------------------+----------------------+
|                        Text                        |      Similarity      |
+====================================================+======================+
+----------------------------------------------------+----------------------+
+----------------------------------------------------+----------------------+
|                       fjeji                        |         nan          |
+====================================================+======================+
+----------------------------------------------------+----------------------+
+----------------------------------------------------+----------------------+
|                       iris1                        |         nan          |
+====================================================+=======

In [214]:
import os
import glob
import re

# le chemin vers tous les fichiers conll

folder_path = glob.glob("corpus-test/*/*.conll")

id_fi = []
lists = []


pat = re.compile("iris[0-9]{3,5}_a.+\.conll$")
pat2 = re.compile("[a-z]\'[a-z]+")

# on parcourt la liste des chemins des fichiers conll
for p in folder_path:

    # l'identifiant d'un fichier correpond à cette regex
    filename = pat.search(p)

    # on récupère chaque id et on le met dans une liste 
    id_fi.append(filename)

    # pour chaque fichier lu

    with open(p, 'r') as files:
        # à chaque passage d'un fichier lu
        lists_fi = []
        for file in files:
            cols = file.rstrip('\n').split('\t')
            if len(cols) == 3:
                # on ajoute la colonnes des lemmes dans une liste
                lists_fi.append(cols[0])
    # puis, on met toutes les listes des lemmes dans une liste globale
    lists.append(lists_fi)
    
#print(lists)

file = open("text.txt", "w")

phrase = ""
for element in lists:
    for mot in element:
        phrase = phrase +" "+ mot
    file.write(phrase)

file.close()
        
        
conversation = phrase.split(".")
print(conversation)

[' Je pense que vous avez donné la réponse dans votre premier message ', " Les textes sont souvent volontairement flous pour laisser la souveraineté d' appréciation aux juges , chaque juge ayant la sienne ", " Il n' y aurait donc pas de réponse formelle à votre question ", ' Bonjour , je viens quérir auprès de vous des conseils et surtout des éclaircissements relatifs au changement de nom pour le motif de " relever un nom qui s\' est illustré sur le plan national " ( art ', ' 61 du Code Civil ) ', " La notion me semble relativement floue , et laisse aux magistrats la possibilité d' apprécier très largement cette notion ", " Outre le peu d' exemples que l' on trouve ici et là ( de Talleyrand , de Tocqueville , etc ", " ) , en existe-t-il d' autres ? Savez -vous si les magistrats estiment par exemple que les personnages politiques ( députés , sénateurs , ministres ) son susceptibles d' entrer dans ce champ ? Y a-t-il des précédents ? Merci d' avance pour vos lumières Pour ce qui est de l

In [171]:
from chatterbot import ChatBot
chatbot = ChatBot("Ron Obvious")

from chatterbot.trainers import ListTrainer

trainer = ListTrainer(chatbot)

trainer.train(conversation)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/duhayon-fontaine/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /Users/duhayon-
[nltk_data]     fontaine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
List Trainer: [####################] 100%


In [220]:
response = chatbot.get_response("Mes parents sont divorcés, puis-je le changer? ")
print(response)

Vu que je suis nouveau , je me présente rapidement : Rémy , étudiant en master à l' ESPE ( afin de devenir professeur des écoles )


In [217]:
import markovify

# Get raw text as string.
with open("text.txt", "rb") as f:
    text = f.read().decode("UTF-8")
    #print(text)

# Build the model.
text_model = markovify.Text(text)

# Print five randomly-generated sentences
for i in range(5):
    print(text_model.make_sentence())

# Print three randomly-generated sentences of no more than 140 characters
for i in range(3):
    print(text_model.make_short_sentence(100))

KeyError: ('___BEGIN__', '___BEGIN__')

In [221]:
#!/usr/bin/python3
# -*- coding: utf-8 -*

# premier test parcours
import os
import glob
import re

# le chemin vers tous les fichiers conll

folder_path = glob.glob("/Users/ferialyahiaoui/Documents/cours/TIM-M2/Groupes/Pretraitement/Corpus/net-iris/*/*/*/*.conll")

#folder_path = glob.glob("/Users/ferialyahiaoui/Documents/cours/*.conll")



id_fi = []
lists = []
dico_q = {} # dico des questions : clé : id_fichier, valeur : liste des lemmes


#pat1 = re.compile('iris([0-9]{3,5})_q([0-9]{1,3})\.conll$')

pat = re.compile("iris[0-9]{3,5}_[q|a].+\.conll$")



# on parcourt la liste des chemins des fichiers conll
for p in folder_path:

	# l'identifiant d'un fichier correpond à cette regex
	filename = pat.search(p)
	if filename:
		id_fi.append(filename.group())
		#print(filename.group())
#print(id_fi)

	# on récupère chaque id et on le met dans une liste 
	
	# pour chaque fichier lu
	
	with open(p, 'r') as files:
		# à chaque passage d'un fichier lu
		lists_w = []

		
		for file in files:
			cols = file.rstrip('\n').split('\t')
			if len(cols) == 3:
				# on ajoute la colonnes des lemmes dans une liste
				lists_w.append(cols[2])
		#print(lists_w)
		# on parcourt la liste des id des fichiers 
		for filename in id_fi:
			# on crée un dic avec comme clé : id_fic et comme valeur : liste de lemmes du fic en question
			dico_q[filename] = lists_w

	# puis, on met toutes les listes des lemmes dans une liste globale
	lists.append(lists_w)
print(lists)
print('\n')
print(dico_q)


[]


{}
